In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time

from oraclai.llm_guide.utils import *
from oraclai.llm_guide.init_utils import *
from oraclai.llm_guide.infer_utils import *
from oraclai.llm_guide.loop_utils import *
from oraclai.llm_guide.mongo_utils import *
from oraclai.llm_guide.manual_ndd import *

/Users/parsaalian/Desktop/All/work/research/parsa-oraclai/oraclai/llm_guide/manual_ndd.py:41: SyntaxWarning: invalid escape sequence '\/'
  'http:\/\/localhost:3000\/default-channel\/products\/.+'
/Users/parsaalian/Desktop/All/work/research/parsa-oraclai/oraclai/llm_guide/manual_ndd.py:45: SyntaxWarning: invalid escape sequence '\/'
  'http:\/\/localhost:3000\/default-channel\/collections\/.+',
/Users/parsaalian/Desktop/All/work/research/parsa-oraclai/oraclai/llm_guide/manual_ndd.py:46: SyntaxWarning: invalid escape sequence '\/'
  'http:\/\/localhost:3000\/default-channel\/categories\/.+'


In [3]:
load_dotenv()

True

In [4]:
action_func_db.delete_many({ 'app': os.getenv("APP_NAME") })
func_db.delete_many({ 'app': os.getenv("APP_NAME") })

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000384'), 'opTime': {'ts': Timestamp(1721690773, 147), 't': 900}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1721690773, 147), 'signature': {'hash': b"f\x1eb\xe29\x08\x1eH\x89'\x01\x026\xf4\xe2\x01\xc9po\xe2", 'keyId': 7363753538167504968}}, 'operationTime': Timestamp(1721690773, 147)}, acknowledged=True)

In [5]:
crawl_context: CrawlContext = CrawlContext()
crawl_context = crawl_context.set_temp_var('config_path', './configs/test.json')

config: dict = read_config(config_path=crawl_context.temp_vars.get('config_path', None))
config_obj: Config = Config.from_dict(config)

if config_obj.base_url is None:
    raise ValueError('base_url is required in config')

crawl_context = crawl_context.set_config(config_obj)

driver = initialize_driver(config_obj)
crawl_context = crawl_context.set_driver(driver)

crawl_context = initialize_variables(crawl_context)

  Reading config from ./configs/test.json
  Initializing driver
  Initializing classes with initial state


In [6]:
driver = initialize_driver(config_obj)
crawl_context.driver = driver

  Initializing driver


# Run

In [7]:
def force_critical(state: State, action: Action) -> bool:
    pair = (state.url, action.get_id())
    return pair in FORBIDDEN_ACTIONS

In [8]:
LOOP_COUNTER = 0
LOOP_LIMIT = 100
FIRST_RUN = True
VISIT_COUNTER = {}

In [9]:
list(map(lambda x: x.url, crawl_context.crawl_queue.items))

['http://localhost:3000/default-channel']

In [10]:
while len(crawl_context.crawl_queue) > 0:
    state: State = crawl_context.crawl_queue.dequeue()
    logger.info(f"Visiting state {state.get_id(StateIdEvaluator.BY_ACTIONS)}")
    crawl_context.state_machine.set_current_state(state)
    
    current_state: State = crawl_context.state_machine.get_current_state()
    current_actions: list[Action] = current_state.get_actions()

    crawl_context.load_state(current_state)

    # logger.info('Extracting state context using LLM')

    # state_context = extract_state_context(
    #     crawl_context,
    #     current_state,
    #     current_state.crawl_path.get_state(-1) if len(current_state.crawl_path) > 0 else None,
    #     current_state.crawl_path.get_action(-1) if len(current_state.crawl_path) > 0 else None,
    # )
    # current_state.set_context(state_context)

    for action in current_actions:
        LOOP_COUNTER += 1
        
        logger.info(f'Executing action {action.element.outerHTML}')
        
        # This variable is true unless an action leads to near-duplicate state.
        # Using this variable we can extract the functionality if even if the action is critical and we wouldn't execute it.
        should_extract_func = True

        is_critical = is_action_critical(action) or force_critical(current_state, action)
        
        if not is_critical:
            if action.get_type().get_value() == 'form':
                values = create_form_filling_values(action)
                action.set_params(values)
            
            action.execute(crawl_context.driver)

            new_actions = []

            for i in range(10):
                try:
                    new_actions: list[Action] = CandidateActionExtractor.extract_candidate_actions(crawl_context.driver)
                    break
                except:
                    time.sleep(0.1)

            if len(new_actions) == 0:
                raise Exception("no new actions")
            
            new_state: State = crawl_context.create_state_from_driver(new_actions)

            VISIT_COUNTER, visit_forbidden_bool = is_visit_forbidden(new_state, VISIT_COUNTER)
            
            if not visit_forbidden_bool and not is_state_in_graph(crawl_context, new_state):
                print('Adding state', new_state.get_id(StateIdEvaluator.BY_ACTIONS))
                crawl_context.crawl_queue.enqueue(new_state)
                crawl_context.state_machine.add_state_from_current_state(new_state, action)
            else:
                should_extract_func = False

        # if should_extract_func:
        #     logger.info(f'Extracting action scenarios: {action.element.outerHTML}')

        #     functionalities = extract_action_functionalities(current_state, action)
        #     if len(functionalities) != 0:
        #         functionality_ids = insert_functionalities(functionalities)
        #         insert_action_functionality(
        #             func_ids=functionality_ids,
        #             state_id=state.get_id(StateIdEvaluator.BY_ACTIONS),
        #             state_url=state.url,
        #             prev_state_id=state.crawl_path.get_state(-1).get_id(StateIdEvaluator.BY_ACTIONS) if len(state.crawl_path) > 0 else None,
        #             action_id=action.get_id(),
        #             prev_action_id=state.crawl_path.get_action(-1).get_id() if len(state.crawl_path) > 0 else None,
        #             action_test_id=action.element.test_id,
        #             action_depth=len(state.crawl_path),
        #             action_type="SINGLE"
        #         )
        
        #     if len(current_state.crawl_path) > 0:
        #         logger.info('Extracting double action scenarios')
        #         functionalities = extract_action_functionalities(current_state, action, current_state.crawl_path.get_action(-1))
        #         if len(functionalities) != 0:
        #             functionality_ids = insert_functionalities(functionalities)
        #             insert_action_functionality(
        #                 func_ids=functionality_ids,
        #                 state_id=state.get_id(StateIdEvaluator.BY_ACTIONS),
        #                 state_url=state.url,
        #                 prev_state_id=state.crawl_path.get_state(-1).get_id(StateIdEvaluator.BY_ACTIONS) if len(state.crawl_path) > 0 else None,
        #                 action_id=action.get_id(),
        #                 prev_action_id=state.crawl_path.get_action(-1).get_id() if len(state.crawl_path) > 0 else None,
        #                 action_test_id=action.element.test_id,
        #                 action_depth=len(state.crawl_path),
        #                 action_type="DOUBLE"
        #             )
                
        #         logger.info('Updating action scores')
    
        #         update_functionality_score(
        #             current_state.crawl_path.get_state(-1),
        #             current_state.crawl_path.get_action(-1),
        #             current_state,
        #             action
        #         )

        #         logger.info('Action scores updated')

        #     logger.info('Marking final functionalities')

        #     mark_final_functionalities(current_state, action)

        #     logger.info('Final actions marked')
        
        crawl_context.load_state(crawl_context.state_machine.get_current_state())

        logger.info("")

crawl_context.driver.quit()

  Visiting state ad57b8d7dbd18aea38be173a085d68c2a8b598462d38062874012d3f3f6e4dae
  Executing action <a aria-label="homepage" clickid="c5-logo" data-testid="c5-logo" href="/default-channel">ACME</a>
  Prompt:
  <a aria-label="homepage" clickid="c5-logo" data-testid="c5-logo" href="/default-channel">ACME</a>
  Response:
  False
  
Adding state ad57b8d7dbd18aea38be173a085d68c2a8b598462d38062874012d3f3f6e4dae
  
  Executing action <a class="border-transparent text-neutral-500 inline-flex items-center border-b-2 pt-px text-sm font-medium hover:text-neutral-700" clickid="c13-navbar-products" data-testid="c13-navbar-products" href="/default-channel/products">All</a>
  Prompt:
  <a class="border-transparent text-neutral-500 inline-flex items-center border-b-2 pt-px text-sm font-medium hover:text-neutral-700" clickid="c13-navbar-products" data-testid="c13-navbar-products" href="/default-channel/products">All</a>
  Response:
  False
  
Adding state dcab579d60141621cd541ec171dba3fbb266e9abdbebfd

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//*[@data-submitid='None']"}
  (Session info: chrome=126.0.6478.183)
Stacktrace:
0   chromedriver                        0x000000010f3b2078 chromedriver + 5169272
1   chromedriver                        0x000000010f3a9f4a chromedriver + 5136202
2   chromedriver                        0x000000010ef2636c chromedriver + 402284
3   chromedriver                        0x000000010ef73740 chromedriver + 718656
4   chromedriver                        0x000000010ef73a01 chromedriver + 719361
5   chromedriver                        0x000000010efb8bc4 chromedriver + 1002436
6   chromedriver                        0x000000010ef96add chromedriver + 862941
7   chromedriver                        0x000000010efb5f57 chromedriver + 991063
8   chromedriver                        0x000000010ef96853 chromedriver + 862291
9   chromedriver                        0x000000010ef665c6 chromedriver + 665030

In [ ]:
print(LOOP_COUNTER)

In [ ]:
for state_id, state_obj in crawl_context.state_machine.state_graph.states.items():
    print(state_id, state_obj.url)

In [14]:
states_converted = {}

for state_id, state_obj in crawl_context.state_machine.state_graph.states.items():
    states_converted[state_id] = {
        'url': state_obj.url,
        'context': state_obj.context,
        'actions': [{
                'type': a.action_type.get_value(),
                'id': a.element.get_id(),
                'outerHTML': clean_children_html(a.element.outerHTML),
                'testId': a.element.test_id
            } for a in state_obj.get_actions()
        ],
        'prev_state': state_obj.crawl_path.get_state(-1).get_id() if len(state.crawl_path) > 0 else None,
        'prev_action': state_obj.crawl_path.get_action(-1).get_id() if len(state.crawl_path) > 0 else None,
    }


adj_list_converted = {}

for state_id, neighbor_list in crawl_context.state_machine.state_graph.adjacency_list.items():
    adj_list_converted[state_id] = {}

    for action_obj, n_state_id in neighbor_list.items():
        adj_list_converted[state_id][action_obj.get_id()] = n_state_id

In [15]:
import json

json.dump(
    {
        'nodes': states_converted,
        'edges': adj_list_converted
    },
    open('./report/saleor.json', 'w+')
)